# Dataset: BeerBot

In [1]:
%pip install beautifulsoup4 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup 
# Documentation for BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests

## Get the data

We will use the recipes from [maischemalzundmehr.de](https://www.maischemalzundmehr.de)<br>
We reduce the recipes to top-fermented (obergärig), because it's easier to brew.<br>
The [search query](https://www.maischemalzundmehr.de/index.php?inhaltmitte=lr&suche_sorte=Sorte+%28egal%29&suche_maische=Maische+%28egal%29&suche_gaerung=oberg%C3%A4rig&suche_order=Sortierung+%28keine%29&suche_begriff=&suche=Suche) returns 1336 recipes. (The images are not up to date, so some numbers vary.)

![maischemalzundmehr.jpg](img/maischemalzundmehr.jpg)


### Web scraping

The results are listed on 122 pages. Through the link preview we can see how we can access all pages.

![pages.gif](img/pages.gif)

In [3]:
# url = https://www.maischemalzundmehr.de/index.php?inhaltmitte=lr&seite=2&suche_gaerung=oberg%C3%A4rig
# We use string formatting to insert the variable {page_number} instead of the actual number.
page_number = 1
url_page = f"https://www.maischemalzundmehr.de/index.php?inhaltmitte=lr&seite={page_number}&suche_gaerung=oberg%C3%A4rig"
print(url_page)

https://www.maischemalzundmehr.de/index.php?inhaltmitte=lr&seite=1&suche_gaerung=oberg%C3%A4rig


Next we will extract all links of one page that lead to the recipes. Through inspection of the html code we can get an identifiyer for all links: the class "rezeptlink":

![rezeptlink.jpg](img/rezeptlink.jpg)

### Extract all ids from one page

In [4]:
# Request html code of page 1:
response = requests.get(url_page)

# Read html code as BeautifulSoup object:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Extract all links that have the attribute class="rezeptlink".
for a in soup.find_all('a', class_="rezeptlink"):
    print(a['href'])

index.php?id=2094&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2093&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2092&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2090&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2089&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2088&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2085&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2084&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2083&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2082&inhaltmitte=rezept&suche_gaerung=obergärig
index.php?id=2080&inhaltmitte=rezept&suche_gaerung=obergärig


In [6]:
# As we need just the id, we can drop the rest.
link = "index.php?id=1535&inhaltmitte=rezept&suche_gaerung=obergärig"
link = link[link.find('id=')+3:link.find('&')]
link

'1535'

In [7]:
# Now we can store this in a list.
ids = []
for a in soup.find_all('a', class_='rezeptlink'):
    link = a['href']
    id_ = link[link.find('id=')+3:link.find('&')]
    ids.append(id_)

In [8]:
print(ids)

['2094', '2093', '2092', '2090', '2089', '2088', '2085', '2084', '2083', '2082', '2080']


In [9]:
# We will wrap the code above into a function.
def get_ids(url):
    ''' Return all ids linked on one page. '''
    # Load html code of page:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    ids_ = [] # temporary list for ids.
    
    for a in soup.find_all('a', class_='rezeptlink'):
        link = a['href']
        id_ = link[link.find('id=')+3:link.find('&')]
        ids_.append(id_)
        
    return ids_

### Loop through all pages and extract all links

In [10]:
ids = []
for i in range(1, 122+1):
    page_number = i
    url_page = f"https://www.maischemalzundmehr.de/index.php?inhaltmitte=lr&seite={page_number}&suche_gaerung=oberg%C3%A4rig"
    # Get ids and add them to the list
    ids += get_ids(url_page)

In [11]:
# As a backup we will store all ids in a text file.
with open('data/beer_ids.txt', 'w') as f:
    for id_ in ids:
        f.write(str(id_)+'\n')

### Normalizing data

If we compare just the first three recipes, we see that all result in a different quantity of beer. In order to use the data for machine learning, we have to normalize it, thus calculate the ingredients for a fixed quantity of beer.

![different_quantities.jpg](img/different_quantities.jpg)

Luckily the website offers a function to export a recipe as JSON.<br>
Url by default:<br>
https://www.maischemalzundmehr.de/export_json.php?id=1544&factoraw=&factorsha=&factorhav=&factorha1=&factorha2=&factorha3=&factorha4=&factorha5=

Inserting 20 instead of 44L for the Nordic Ale results in a changed URL:<br>
https://www.maischemalzundmehr.de/export_json.php?id=1544&factoraw=20&factorsha=65&factorhav=13.2&factorha1=3.4&factorha2=13.2&factorha3=11.1&factorha4=&factorha5=

Luckily it's enough to use the first parameter to receive the recipt in JSON format.<br>
https://www.maischemalzundmehr.de/export_json.php?id=1544&factoraw=20

In [19]:
# Download a recipe with requests.
response = requests.get('https://www.maischemalzundmehr.de/export_json.php?id=1544&factoraw=20')
with open('test.json', 'w', encoding='utf-8') as f:
    f.write(response.text)

### Download all recipes in JSON-format

In [21]:
from tqdm import tqdm  # To visualize the progress

for id_ in tqdm(ids):
    response = requests.get(f"https://www.maischemalzundmehr.de/export_json.php?id={id_}&factoraw=20")
    with open(f"data/beer_recipes/{id_.zfill(4)}'.json", 'w', encoding='utf-8') as f:
        f.write(response.text)

100%|█████████████████████████████████████████████████| 1336/1336 [05:07<00:00,  4.34it/s]


## Create the dataset

Every recipe is stored in a separate json file. We'll use this to create a JSONL file ('L' = line. Every line holds a separate JSON object instead of having all recipes in one JSON object. JSONL is useful for very large datasets (so not for this one, but we'll use it nevertheless for convenience).

In [22]:
import os
import json

def read_json_files_from_directory(directory_path):
    """
    Reads all JSON files from the specified directory and returns a list of JSON objects.
    """
    json_objects = []
    filenames = os.listdir(directory_path)
    filenames.sort()
    for filename in filenames:
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                json_objects.append(json.load(file))
    return json_objects

def write_jsonl_file(json_objects, output_path):
    """
    Writes a list of JSON objects to a JSON Lines (JSONL) file.
    """
    with open(output_path, 'w', encoding='utf-8') as file:
        for obj in json_objects:
            json.dump(obj, file)
            file.write('\n')
            
recipes_json = read_json_files_from_directory('data/beer_recipes')
write_jsonl_file(recipes_json, 'data/beer.jsonl')

In [23]:
# Create a new JSONL file
with open('data/beer.jsonl', 'r', encoding='utf8') as f:
    # Loop through the list of recipes
    recipes = f.readlines()

In [24]:
for recipe in recipes[:3]:
    print(recipe)

{"Rezeptquelle": "www.maischemalzundmehr.de", "ExportVersion": "2.0", "Name": "Alt 43", "Datum": "14.02.2011", "Sorte": "Altbier", "Autor": "muldengold", "Ausschlagwuerze": 20, "Stammwuerze": 14.5, "Bittere": 25, "Farbe": 35, "Alkohol": 6, "Kurzbeschreibung": "Malziges Altbier mit dezentem Hopfenaroma und -geschmack", "Malze": [{"Name": "M&uuml;nchner Malz", "Menge": 2.91, "Einheit": "kg"}, {"Name": "Pilsner Malz", "Menge": 1.09, "Einheit": "kg"}, {"Name": "R&ouml;stmalz", "Menge": 36, "Einheit": "g"}], "Maischform": "infusion", "Hauptguss": 13.454545454545455, "Einmaischtemperatur": 50, "Rasten": [{"Temperatur": 52, "Zeit": 15}, {"Temperatur": 65, "Zeit": 70}, {"Temperatur": 78, "Zeit": 10}], "Abmaischtemperatur": 78, "Nachguss": 16.727272727272727, "Kochzeit_Wuerze": 70, "Hopfenkochen": [{"Sorte": "Saaz", "Menge": 18, "Alpha": 4.4, "Zeit": 70, "Typ": "Vorderwuerze"}, {"Sorte": "Northern Brewer", "Menge": 9, "Alpha": 10, "Zeit": 60, "Typ": "Standard"}, {"Sorte": "Saaz", "Menge": 9, "A

### Conversation

In [25]:
# We'll use this function from the 
def str_to_dict(user, assistant):
    # create a list with the entries as dictionaries
    conversation_data = [{'role':'user', 'content':user}, {'role':'assistant', 'content':assistant}]
    # create a dictionary with key 'conversations' and add the list as value
    dictionary = {'conversations':conversation_data}
    return dictionary

def extract_field(json_string, field):
    """
    Extracts the value for a field from a JSON-formatted string.

    :param json_string: A string containing JSON data.
    :return: The value of the field, or None if the field does not exist.
    """
    try:
        # Parse the JSON string into a Python dictionary
        data = json.loads(json_string)

        # Extract the value for the "kurzbeschreibung" field
        result = data.get(field)

        return result
    except json.JSONDecodeError:
        # Handle the case where the JSON string is invalid
        print("Invalid JSON string")
        return None


dataset = []

with open('data/beer.jsonl', 'r', encoding='utf8') as f:
    data = f.readlines()
    
for sample in data:
    # Extract the user content
    user = extract_field(sample, 'Kurzbeschreibung')
    if user is not None:
        # The answer of the assistant will be the whole JSON object
        dict_entry = str_to_dict(user, sample) 
        dataset.append(dict_entry)

In [26]:
len(dataset)

1297

In [27]:
dataset[0]

{'conversations': [{'role': 'user',
   'content': 'Malziges Altbier mit dezentem Hopfenaroma und -geschmack'},
  {'role': 'assistant',
   'content': '{"Rezeptquelle": "www.maischemalzundmehr.de", "ExportVersion": "2.0", "Name": "Alt 43", "Datum": "14.02.2011", "Sorte": "Altbier", "Autor": "muldengold", "Ausschlagwuerze": 20, "Stammwuerze": 14.5, "Bittere": 25, "Farbe": 35, "Alkohol": 6, "Kurzbeschreibung": "Malziges Altbier mit dezentem Hopfenaroma und -geschmack", "Malze": [{"Name": "M&uuml;nchner Malz", "Menge": 2.91, "Einheit": "kg"}, {"Name": "Pilsner Malz", "Menge": 1.09, "Einheit": "kg"}, {"Name": "R&ouml;stmalz", "Menge": 36, "Einheit": "g"}], "Maischform": "infusion", "Hauptguss": 13.454545454545455, "Einmaischtemperatur": 50, "Rasten": [{"Temperatur": 52, "Zeit": 15}, {"Temperatur": 65, "Zeit": 70}, {"Temperatur": 78, "Zeit": 10}], "Abmaischtemperatur": 78, "Nachguss": 16.727272727272727, "Kochzeit_Wuerze": 70, "Hopfenkochen": [{"Sorte": "Saaz", "Menge": 18, "Alpha": 4.4, "Zei

## Save the dataset

In [31]:
write_jsonl_file(dataset, 'data/beer_conversations.jsonl')

## Load the dataset

In [33]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='data/beer_conversations.jsonl', split='train')

In [34]:
dataset[:3]

{'conversations': [[{'role': 'user',
    'content': 'Malziges Altbier mit dezentem Hopfenaroma und -geschmack'},
   {'role': 'assistant',
    'content': '{"Rezeptquelle": "www.maischemalzundmehr.de", "ExportVersion": "2.0", "Name": "Alt 43", "Datum": "14.02.2011", "Sorte": "Altbier", "Autor": "muldengold", "Ausschlagwuerze": 20, "Stammwuerze": 14.5, "Bittere": 25, "Farbe": 35, "Alkohol": 6, "Kurzbeschreibung": "Malziges Altbier mit dezentem Hopfenaroma und -geschmack", "Malze": [{"Name": "M&uuml;nchner Malz", "Menge": 2.91, "Einheit": "kg"}, {"Name": "Pilsner Malz", "Menge": 1.09, "Einheit": "kg"}, {"Name": "R&ouml;stmalz", "Menge": 36, "Einheit": "g"}], "Maischform": "infusion", "Hauptguss": 13.454545454545455, "Einmaischtemperatur": 50, "Rasten": [{"Temperatur": 52, "Zeit": 15}, {"Temperatur": 65, "Zeit": 70}, {"Temperatur": 78, "Zeit": 10}], "Abmaischtemperatur": 78, "Nachguss": 16.727272727272727, "Kochzeit_Wuerze": 70, "Hopfenkochen": [{"Sorte": "Saaz", "Menge": 18, "Alpha": 4.4, 